In [100]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from fancyimpute import KNN
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import KBinsDiscretizer

import matplotlib.pyplot as plt
import pydotplus

import collections
from sklearn.model_selection import StratifiedKFold

import plotly.express as px
import plotly

import seaborn as sns

import plotly.offline
import plotly.graph_objs as go


In [3]:
# Read all samples from the file
data_kihd = pd.read_excel('kihd_may_2019.xlsx')

In [4]:
# -----------------------------------------
# Keep the original data in 'data_kihd' and use its copy 'data_kihd_preprocessed' instead
# -----------------------------------------
data_kihd_preprocessed = data_kihd.copy()

# Drop two variables (dates of visits)
data_kihd_preprocessed = data_kihd_preprocessed.drop(['tpvm2', 'tpnr2'], axis = 1)

# Correct the zero-level for the following variables:
wrong_zero = ['v0563', 'v0565', 'v0567', 'v0569', 'v0571', 'v0573', 'v0575', 'v0577', 'v0579', 'v0607', 'v0609', 'v0613',
              'v0621', 'v0623', 'v0625', 'v0627', 'v0629', 'v0631', 'v0633', 'v0635', 'v0637', 'v0639', 'v0643', 'v0645', 
              'v0647', 'v0649', 'v0651', 'v0653']
# Zeros are at the wrong end of the scale
# Change zeros to (max+1)
for col in wrong_zero:
    data_kihd_preprocessed.loc[:, col] = [np.max(data_kihd_preprocessed.loc[:, col]) + 1 if value == 0.0 else value for value in data_kihd_preprocessed.loc[:, col]]

print(data_kihd_preprocessed.shape)
# -----------------------------------------
# Turn the categorical variables indo dummies
# -----------------------------------------
categorical_variables=['au0136','au0153','ek0115','ek0119','ka0118','mi0205','mi0207','mi0208','mi0209',
                       'mi0210','mi0211','mi0212','mi0213','mi0214','v0145','v0146','v0157','v0158','v0161',
                       'v0172','v0247','v0248','v0665','v0721','v0724','u1307']

for col in categorical_variables:
    if col in data_kihd_preprocessed.columns:
        new_dummies=pd.get_dummies(data_kihd_preprocessed[col], dummy_na=False)
        my_list = new_dummies.columns.values
        string = col+"_"
        my_new_list = [string + str(x) for x in my_list]
        new_dummies.columns = my_new_list
        data_kihd_preprocessed = data_kihd_preprocessed.drop(col, axis=1)       
        data_kihd_preprocessed = data_kihd_preprocessed.join(new_dummies)
                 
# Outcomes
kihd_outcomes = data_kihd_preprocessed.loc[:, ['tutknro', 'chdb16', 'chdb16d', 'amif16', 'amif16d', 'amig16', 'amig16d',
       'amim16', 'amim16d', 'all16', 'all16d', 'cvd16', 'cvd16d', 'syd14', 'alzm14', 'vp14', 'kol14', 'diab14',
       'ncvd16', 'ncvd16d', 'cv15', 'cv15d', 'all15', 'stro15', 'cvd15', 'chd15', 'amib15', 'isth15', 'hsth15', 
                                'db15', 'can15', 'canc15', 'ast15', 'copd15', 'dema15', 'finchd18', 'finchd18d']]

# Predictors
kihd_predictors = data_kihd_preprocessed.drop(['tutknro', 'chdb16', 'chdb16d', 'amif16', 'amif16d', 'amig16', 'amig16d',
       'amim16', 'amim16d', 'all16', 'all16d', 'cvd16', 'cvd16d', 'syd14', 'alzm14', 'vp14', 'kol14', 'diab14',
       'ncvd16', 'ncvd16d', 'cv15', 'cv15d', 'all15', 'stro15', 'cvd15', 'chd15', 'amib15', 'isth15', 'hsth15', 
                                'db15', 'can15', 'canc15', 'ast15', 'copd15', 'dema15', 'finchd18', 'finchd18d'], axis = 1)

# Separate genes and phenotypes
genes_start = list(kihd_predictors.columns.values).index('FEDER2HH'.lower())
genes_end = list(kihd_predictors.columns.values).index('CATAETT'.lower())

# Genes only
kihd_genes = kihd_predictors.iloc[:, genes_start:(genes_end+1)].copy()
print("KIHD with genes only: {}".format(kihd_genes.shape))

# Phenotypes
kihd_phenotypes = kihd_predictors.drop(kihd_predictors.columns.values[genes_start:genes_end+1], axis = 1)
print("KIHD with phenotypes only: {}".format(kihd_phenotypes.shape))

(2682, 992)
KIHD with genes only: (2682, 96)
KIHD with phenotypes only: (2682, 977)


In [ ]:
# -----------------------------------------
# Remove predictors and subjects based on the number of missing values in 'kihd_phenotypes'
# -----------------------------------------
# Remove variables (columns) containing more than 5% of missing values
threshold_columns = kihd_phenotypes.shape[0]-round(0.05*kihd_phenotypes.shape[0])
kihd_phenotypes = kihd_phenotypes.dropna(axis=1, thresh=threshold_columns) 

print("Filter out predictors with more than 5% of missing values ...")
print("Dataset size: {rows}x{cols}".format(rows = kihd_phenotypes.shape[0], cols = kihd_phenotypes.shape[1]))
print("-------------------------------------")

# Remove subjects (rows) with more than 5% of missing values
threshold_rows = kihd_phenotypes.shape[1]-round(0.05*kihd_phenotypes.shape[1])
kihd_phenotypes = kihd_phenotypes.dropna(axis=0, thresh=threshold_rows)

print("Filter out rows with more than 5% of missing values ...")
print("Dataset size: {rows}x{cols}".format(rows = kihd_phenotypes.shape[0], cols = kihd_phenotypes.shape[1]))
print("-------------------------------------")

# Remove subjects in genes and outcomes correspondingly
kihd_genes = kihd_genes.loc[kihd_phenotypes.index, :]
kihd_outcomes = kihd_outcomes.loc[kihd_phenotypes.index, :]

# Reset indices in all data frames after removing subjects 
kihd_genes = kihd_genes.reset_index(drop=True)
kihd_outcomes = kihd_outcomes.reset_index(drop=True)
kihd_phenotypes = kihd_phenotypes.reset_index(drop=True)

In [ ]:
# -----------------------------------------
# Fill gaps in 'kihd_phenotypes' with kNN
# -----------------------------------------

# Scale predictors before applying the NN-based method
scaler=MinMaxScaler().fit(kihd_phenotypes)
kihd_phenotypes_scaled=scaler.transform(kihd_phenotypes)
kihd_phenotypes_scaled_filled_knn = KNN(k=1).fit_transform(kihd_phenotypes_scaled)

# Inverse scaling to original ranges
kihd_phenotypes=pd.DataFrame(scaler.inverse_transform(kihd_phenotypes_scaled_filled_knn), columns=kihd_phenotypes.columns.values)

In [7]:
# -----------------------------------------
# Handle competing risks
# -----------------------------------------
# Remove subjects died because of any non-cardiovascular reason within the prediction horizon

prediction_horizon = 35 * 365
kihd_outcomes = kihd_outcomes.drop(kihd_outcomes[ (kihd_outcomes.loc[:, 'ncvd16'] == 1) & 
                                                 (kihd_outcomes.loc[:, 'ncvd16d'] <= prediction_horizon)].index, axis=0)
# Remove subjects in kihd_genes and kihd_phenotypes correspondingly
kihd_genes = kihd_genes.loc[kihd_outcomes.index, :]
kihd_phenotypes = kihd_phenotypes.loc[kihd_outcomes.index, :]

# Reset indices in all data frames after removing subjects 
kihd_genes = kihd_genes.reset_index(drop=True)
kihd_outcomes = kihd_outcomes.reset_index(drop=True)
kihd_phenotypes = kihd_phenotypes.reset_index(drop=True)

In [ ]:
# -----------------------------------------
# Collect statistics for subjects in multiple runs of k-fold cross-validation
# -----------------------------------------

statistics_cv = pd.DataFrame({'TP': [0]*kihd_phenotypes.shape[0], 'TN': [0]*kihd_phenotypes.shape[0], 
                                    'FP': [0]*kihd_phenotypes.shape[0], 'FN': [0]*kihd_phenotypes.shape[0]})

outcome = 'cvd16'

data_x = kihd_phenotypes.copy()
data_y = kihd_outcomes[[outcome]].values.ravel()

true_y_test = np.array([])
predicted_y_test = np.array([])

true_y_train = np.array([])
predicted_y_train = np.array([])

selected_columns = []

runs = 50
selected_features = 0

for r in range(runs):
    skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)
    for train_index, test_index in skf.split(data_x, data_y):
        data_x_train, data_x_test = data_x.iloc[train_index].copy(), data_x.iloc[test_index].copy()
        y_train, y_test = data_y[train_index], data_y[test_index]
        
        #normalize
        scaler = MinMaxScaler().fit(data_x_train)
        data_x_train = scaler.transform(data_x_train)
        data_x_test = scaler.transform(data_x_test)
        
        #train the model
        model = LogisticRegression(penalty="l1", max_iter=500, solver="liblinear", C=0.15)
        model.fit(data_x_train, y_train)
        
        coef = np.abs(np.array(model.coef_))
        selected_columns.extend(list(kihd_phenotypes.columns[np.where(coef[0] > 0.0)]))
        
        selected_features += len(list(kihd_phenotypes.columns[np.where(coef[0] > 0.0)]))/(runs*5)
        
        class_counts = np.unique(y_train, return_counts = True)[1]
        threshold_logit = np.min(class_counts)/np.sum(class_counts)
        
        #apply to test
        y_prob_test = model.predict_proba(data_x_test)
        predictions_test = [0 if risk[1] < threshold_logit else 1 for risk in y_prob_test]
        predicted_y_test = np.append(predicted_y_test, predictions_test)
        true_y_test = np.append(true_y_test, y_test)
        
        #apply to train
        y_prob_train = model.predict_proba(data_x_train)
        predictions_train = [0 if risk[1] < threshold_logit else 1 for risk in y_prob_train]
        predicted_y_train = np.append(predicted_y_train, predictions_train)
        true_y_train = np.append(true_y_train, y_train)
        
        for i, ind in enumerate(test_index):
            if predictions_test[i] == y_test[i] == 1:
                statistics_cv.loc[ind, 'TP'] += 1
            elif predictions_test[i] == y_test[i] == 0:
                statistics_cv.loc[ind, 'TN'] += 1
            elif predictions_test[i] != y_test[i] == 0:
                statistics_cv.loc[ind, 'FP'] += 1
            elif predictions_test[i] != y_test[i] == 1:
                statistics_cv.loc[ind, 'FN'] += 1 

selected_columns = list(set(selected_columns))
print(len(selected_columns))
print("Test data")
print("Accuracy: {}".format(accuracy_score(true_y_test, predicted_y_test)))
M = confusion_matrix(true_y_test, predicted_y_test)
print(M)
print("TPR: {}".format(M[1,1]/(M[1,0] + M[1,1])))
print("TNR: {}".format(M[0,0]/(M[0,0] + M[0,1])))

print("")
print("Training data")
print("Accuracy: {}".format(accuracy_score(true_y_train, predicted_y_train)))
M = confusion_matrix(true_y_train, predicted_y_train)
print(M)
print("TPR: {}".format(M[1,1]/(M[1,0] + M[1,1])))
print("TNR: {}".format(M[0,0]/(M[0,0] + M[0,1])))

statistics_cv.loc[:, 'accuracy'] = (statistics_cv.loc[:, 'TP'] + statistics_cv.loc[:, 'TN']) / statistics_cv.sum(axis = 1)

# Keep variables selected by the model at least once
kihd_phenotypes = kihd_phenotypes.loc[:, selected_columns]

In [39]:
# -----------------------------------------
# Create intervals or get levels of variables
# -----------------------------------------

def discretize_kmeans(data):
    
    data_discretized = pd.DataFrame(columns = data.columns)
    discrete_levels = {}
    levels_number = []
    
    for i, col in enumerate(data.columns):
        
        unique_levels = np.unique(data.loc[:, col])
        
        if len(unique_levels) <= 2:
            data_discretized.loc[:, col] = data.loc[:, col]
            discrete_levels[col] = [(lvl, lvl) for lvl in unique_levels]
            levels_number.append(2)
            
        elif 2 < len(unique_levels) <= 7:           
            data_discretized.loc[:, col] = [np.where(unique_levels == item)[0] for item in data.loc[:, col]]
            discrete_levels[col] = [(i, lvl) for i, lvl in enumerate(unique_levels)]
            levels_number.append(len(unique_levels))
          
        else:
            n_bins=4
            discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='kmeans')
            discretizer.fit(data.loc[:, col].values.reshape(-1, 1))
            levels_kmeans = discretizer.transform(data.loc[:, col].values.reshape(-1, 1)).ravel()
            data_discretized.loc[:, col] = levels_kmeans
            
            levels_number.append(len(np.unique(levels_kmeans)))
            discrete_levels[col] = []
            for j in range(len(np.unique(levels_kmeans))):
                interval = np.unique(data.loc[np.where(levels_kmeans == j)[0], col])
                discrete_levels[col].append((j, '[{}, {}]'.format(min(interval), max(interval))))
            
    return data_discretized, discrete_levels, levels_number

# -----------------------------------------
data_discretized, discrete_levels, levels_number = discretize_kmeans(kihd_phenotypes)

In [48]:
# -----------------------------------------
# Define an objective function
# -----------------------------------------
def fintess(x):
    return kihd_objective(x, levels_number.copy(), data_discretized.copy())


def kihd_objective(x, levels_number, data_discretized):

    bool_check_colomns = [True if x[i] < lvl else False for i, lvl in enumerate(levels_number)]

    data_discretized_selected_cols = data_discretized.loc[:, bool_check_colomns]
    levels_number_selected_cols = np.array(levels_number)[bool_check_colomns]
    x_selected_cols = list(np.array(x)[bool_check_colomns])

    if len(x_selected_cols) == 0:
        return [100.0, 100.0, len(bool_check_colomns)] # if tpr and tnr are minimized 
        #return [0.0, 0.0, len(bool_check_colomns)] # if tpr and tnr are maximized 
    
    selected = [True if x_selected_cols == list(row) else False for row in data_discretized_selected_cols.values]
    selected = pd.Series(selected)

    number_selected = statistics_cv.loc[selected, :].shape[0]

    if number_selected == 0:
        return [100.0, 100.0, len(selected.values)] # if tpr and tnr are minimized 
        #return [0.0, 0.0, len(selected.values)] # if tpr and tnr are maximized 
    
    sick = (statistics_cv.loc[selected].sum()['TP'] + statistics_cv.loc[selected].sum()['FN'])
    healthy = (statistics_cv.loc[selected].sum()['TN'] + statistics_cv.loc[selected].sum()['FP'])
    
    tpr = 100 * statistics_cv.loc[selected].sum()['TP']/sick if sick!=0 else 0
    tnr = 100 * statistics_cv.loc[selected].sum()['TN']/healthy if healthy!=0 else 0

    return [tpr, tnr, len(selected.values)-number_selected] # if tpr and tnr are minimized
    #return [100-tpr, 100-tnr, len(selected.values)-number_selected] # if tpr and tnr are maximized  

In [44]:
# -----------------------------------------
# Evaluate final solutions
# -----------------------------------------

def solution_evaluation(x, levels_number, data_discretized):

    bool_check_colomns = [True if x[i] < lvl else False for i, lvl in enumerate(levels_number)]
        
    data_discretized_selected_cols = data_discretized.loc[:, bool_check_colomns]
    levels_number_selected_cols = np.array(levels_number)[bool_check_colomns]
    x_selected_cols = list(np.array(x)[bool_check_colomns])

    if len(x_selected_cols) == 0:
        return {'variables': '',
            'tpr': 0.0, 
            'tnr': 0.0, 
            'number': 0,
            'indices': np.nan}
    
    selected = [True if x_selected_cols == list(row) else False for row in data_discretized_selected_cols.values]
    selected = pd.Series(selected)
    
    selected_indices = list(statistics_cv.loc[selected].index)

    number_selected = statistics_cv.loc[selected, :].shape[0]

    if number_selected == 0:
        return {'variables': '',
            'tpr': 0.0, 
            'tnr': 0.0, 
            'number': 0,
            'indices': np.nan}
    
    sick = (statistics_cv.loc[selected].sum()['TP'] + statistics_cv.loc[selected].sum()['FN'])
    healthy = (statistics_cv.loc[selected].sum()['TN'] + statistics_cv.loc[selected].sum()['FP'])
    
    tpr = 100 * statistics_cv.loc[selected].sum()['TP']/sick if sick!=0 else 0
    tnr = 100 * statistics_cv.loc[selected].sum()['TN']/healthy if healthy!=0 else 0
 
    group_variables = ['{}: {}  '.format(col, discrete_levels[col][x_selected_cols[i]]) for i, col in enumerate(data_discretized_selected_cols.columns.values)]
    return {'variables': group_variables,
            'tpr': tpr, 
            'tnr': tnr, 
            'number': number_selected,
            'indices': selected_indices}


In [45]:
# -----------------------------------------
# Generate an initial population
# -----------------------------------------

max_border_of_zero_elements = 2**np.ceil(np.log2(np.array(levels_number) + 1)) - 1
min_border_of_zero_elements = np.array(levels_number)


def generate_int_solution():
    indv_int = []
    for i in range(ndim):

        gene_int_zero = np.random.randint(min_border_of_zero_elements[i], max_border_of_zero_elements[i] + 1, size=1)[0]
        gene_int_not_zero = np.random.randint(min_border_of_zero_elements[i], size=1)[0]

        indv_int.append(choice([gene_int_not_zero, gene_int_zero], 1, p = [0.05, 0.95])[0])
        
    return indv_int


In [ ]:
# -----------------------------------------
# Run NSGAIII
# -----------------------------------------

from platypus.problems import Problem
from platypus.algorithms import NSGAII, NSGAIII
from platypus.types import Binary, Integer
from platypus.core import Solution
from numpy.random import choice
from platypus.operators import InjectedPopulation
from platypus.core import nondominated, Archive
from platypus.tools import int2bin, bin2gray, bin2int, gray2bin

nondominated_solutions = Archive()
solutions_list = []

ndim = len(levels_number)

for r in range(1):
    problem = Problem(len(levels_number), 3)
    problem.types = [Integer(0, 2**np.ceil(np.log2(max_value + 1)) - 1) for max_value in levels_number]
           
    problem.function = fintess

    problem.directions[0] = Problem.MINIMIZE
    problem.directions[1] = Problem.MINIMIZE
    problem.directions[2] = Problem.MINIMIZE

    divisions_outer=20
    algorithm = NSGAIII(problem, divisions_outer=divisions_outer)
    
    population_size = algorithm.population_size
      
    init_pop = [Solution(problem) for i in range(population_size)]
    pop_indiv = []
    for i in range(population_size):
        indv_int = generate_int_solution()
        indv = []
        for j in range(ndim):
            indv.append(bin2gray(int2bin(indv_int[j], np.ceil(np.log2(levels_number[j] + 1)))))
        pop_indiv.append(indv)
    
    for i in range(population_size):
        init_pop[i].variables = pop_indiv[i]


    algorithm = NSGAIII(problem, divisions_outer=divisions_outer, generator=InjectedPopulation(init_pop))
    generations = 5
    algorithm.run(population_size * generations)

    solutions_list.extend(algorithm.result)
    

In [52]:
# -----------------------------------------
# Save all the rules generated
# -----------------------------------------

rules = pd.DataFrame(columns = ['variables', 'tpr', 'tnr', 'number', 'indices'])
for s in solutions_list:
    int_sol = [bin2int(gray2bin(s.variables[i])) for i in range(ndim)]
    rules = rules.append(solution_evaluation(int_sol, levels_number, data_discretized), ignore_index=True) 
    
rules.loc[:, 'variables_str'] = [''.join(rules.loc[i, 'variables']) for i in range(rules.shape[0])]

rules_unique = rules.iloc[np.where(~pd.DataFrame(rules[['variables_str']]).duplicated().values == True)[0]].reset_index(drop=True)

rules_unique.to_csv('lasso_c0.15_runs50_min.csv', index = False) # if tpr and tnr are minimized
#rules_unique.to_csv('lasso_c0.15_runs50_min.csv', index = False) # if tpr and tnr are maximized

RULE PROCESSING
===============

In [55]:
# -----------------------------------------
# If there are equal sets, remove
# -----------------------------------------
def remove_equal_sets(nondominated_objectives_reduced):

    equal_sets = dict((ind,[]) for ind in nondominated_objectives_reduced.index)

    for ind1 in nondominated_objectives_reduced.index:

        for ind2 in nondominated_objectives_reduced.index[(ind1+1):]:
            set1 = set(nondominated_objectives_reduced.loc[ind1, 'indices'])
            set2 = set(nondominated_objectives_reduced.loc[ind2, 'indices'])

            if set1.issubset(set2) and set2.issubset(set1):
                equal_sets[ind1].append(ind2)
                equal_sets[ind2].append(ind1)

    # remove duplicates
    for ind1 in nondominated_objectives_reduced.index:
        if len(equal_sets[ind1]) != 0:
            #equal_sets[ind1].append(ind1)
            groups = equal_sets[ind1]

            for gr in groups:
                set1 = set(nondominated_objectives_reduced.loc[ind1, 'variables'])
                set2 = set(nondominated_objectives_reduced.loc[gr, 'variables'])

                nondominated_objectives_reduced.loc[ind1, 'variables'].extend(list(set2 - set1))

            nondominated_objectives_reduced = nondominated_objectives_reduced.drop(groups, axis = 0)

            for ind2 in equal_sets[ind1]:
                equal_sets[ind2] = []

            equal_sets[ind1] = []

    nondominated_objectives_reduced = nondominated_objectives_reduced.reset_index(drop=True)
    
    return nondominated_objectives_reduced


In [65]:
# -----------------------------------------
# Read rules from files and keep only unique ones
# -----------------------------------------

from ast import literal_eval
objectives_reduced_max = pd.read_csv('lasso_c0.15_runs50_max.csv', sep = ',', converters={'variables': literal_eval, 'indices': literal_eval})
objectives_reduced_min = pd.read_csv('lasso_c0.15_runs50_min.csv', sep = ',', converters={'variables': literal_eval, 'indices': literal_eval})

objectives_reduced_max = objectives_reduced_max.iloc[np.where(~pd.DataFrame(objectives_reduced_max[['variables_str']]).duplicated().values == True)[0]].reset_index(drop=True)
objectives_reduced_min = objectives_reduced_min.iloc[np.where(~pd.DataFrame(objectives_reduced_min[['variables_str']]).duplicated().values == True)[0]].reset_index(drop=True)

objectives_reduced_max = objectives_reduced_max.drop('variables_str', axis = 1)
objectives_reduced_min = objectives_reduced_min.drop('variables_str', axis = 1)

objectives_reduced_max = remove_equal_sets(objectives_reduced_max)
objectives_reduced_min = remove_equal_sets(objectives_reduced_min)


all_groups = pd.DataFrame()
all_groups = pd.concat([objectives_reduced_max.copy(), objectives_reduced_min.copy()]).reset_index(drop=True)

all_groups = remove_equal_sets(all_groups)

In [75]:
# -----------------------------------------
# Estimate accuracy for each rule
# -----------------------------------------
average_accuracy = statistics_cv.loc[:, 'accuracy'].mean()
all_groups.loc[:, 'accuracy'] = [ statistics_cv.loc[all_groups.loc[i, 'indices'], 'accuracy'].mean() for i in range(all_groups.shape[0])]
all_groups.loc[:, 'level'] = [ 'lower' if all_groups.loc[i, 'accuracy'] < average_accuracy else 'higher' for i in range(all_groups.shape[0])] 

In [76]:
# -----------------------------------------
# Select rules using thresholds
# -----------------------------------------
tpr_tnr_difference = (all_groups.loc[:, ['tpr', 'tnr']].max(axis=1) - all_groups.loc[:, ['tpr', 'tnr']].min(axis=1))/all_groups.loc[:, ['tpr', 'tnr']].max(axis=1) < 0.1
min_group_size = all_groups.loc[:, 'number'] >= 30

max_acc_threshold = 0.95
min_acc_threshold = 0.50
all_groups_reduced = all_groups.loc[ min_group_size & tpr_tnr_difference & ( (all_groups.loc[:, 'accuracy'] >= max_acc_threshold) | (all_groups.loc[:, 'accuracy'] <= min_acc_threshold) ) ].reset_index(drop=True)

In [77]:
# -----------------------------------------
# Remove subsets
# -----------------------------------------

def remove_subsets(nondominated_objectives_reduced):
    # if there are subsets
    subsets = dict((ind,[]) for ind in nondominated_objectives_reduced.index)

    for ind1 in nondominated_objectives_reduced.index:

        for ind2 in nondominated_objectives_reduced.index:
            set1 = set(nondominated_objectives_reduced.loc[ind1, 'indices'])
            set2 = set(nondominated_objectives_reduced.loc[ind2, 'indices'])

            if set1.issubset(set2) and ind1!=ind2:
                subsets[ind1].append(ind2)

    # remove subsets
    for ind1 in nondominated_objectives_reduced.index:
        if len(subsets[ind1]) != 0:
            nondominated_objectives_reduced = nondominated_objectives_reduced.drop(ind1, axis = 0)

        for ind2 in nondominated_objectives_reduced.index:
            if ind1 in subsets[ind2]:
                subsets[ind2].remove(ind1) 

            subsets[ind1] = []
            
    return nondominated_objectives_reduced.reset_index(drop=True)

all_groups_reduced = remove_subsets(all_groups_reduced)

In [78]:
# -----------------------------------------
# Check which rules (for easy or diffucult cases) cover each subject
# -----------------------------------------
def get_subjects_in_groups(all_groups_reduced):
    
    subjects_in_groups = pd.DataFrame({'lower': [0]*len(kihd_phenotypes), 'higher': [0]*len(kihd_phenotypes)})

    for i in range(all_groups_reduced.shape[0]):
        subjects_in_groups.loc[all_groups_reduced.loc[i, 'indices'], all_groups_reduced.loc[i, 'level']] += 1

    subjects_in_groups.loc[:, 'accuracy'] = statistics_cv.loc[:, 'accuracy']
    
    return subjects_in_groups

subjects_in_groups = get_subjects_in_groups(all_groups_reduced)

In [79]:
subjects_in_groups.loc[(subjects_in_groups.loc[:, 'lower'] == 0) & (subjects_in_groups.loc[:, 'higher'] == 0)].shape

(751, 3)

In [80]:
# -----------------------------------------
# Get statistics for the chosen thresholds
# -----------------------------------------
def get_cases_number(subjects_in_groups):
        
    easy_cases = 0
    difficult_cases = 0
    mixed_cases = 0
    not_covered_cases = 0

    easiness = []
    
    accuracy_not_covered = 0.
    accuracy_mixed = 0.
    accuracy_easy = 0.
    accuracy_difficult = 0.

    for i in range(subjects_in_groups.shape[0]):
        if subjects_in_groups.loc[i, 'lower'] == 0 and subjects_in_groups.loc[i, 'higher'] == 0:
            not_covered_cases += 1
            easiness.append('not covered')
            accuracy_not_covered += subjects_in_groups.loc[i, 'accuracy']
        else:
            if subjects_in_groups.loc[i, 'lower'] > 0 and subjects_in_groups.loc[i, 'higher'] > 0:
                mixed_cases += 1
                easiness.append('ambiguous case')
                accuracy_mixed += subjects_in_groups.loc[i, 'accuracy']
            elif subjects_in_groups.loc[i, 'lower'] > 0 and subjects_in_groups.loc[i, 'higher'] == 0:
                difficult_cases += 1
                easiness.append('difficult case')
                accuracy_difficult += subjects_in_groups.loc[i, 'accuracy']
            elif subjects_in_groups.loc[i, 'lower'] == 0 and subjects_in_groups.loc[i, 'higher'] > 0:
                easy_cases += 1
                easiness.append('easy case')
                accuracy_easy += subjects_in_groups.loc[i, 'accuracy']

    if easy_cases>0:
        accuracy_easy = accuracy_easy/easy_cases
        
    if difficult_cases>0:
        accuracy_difficult = accuracy_difficult/difficult_cases
        
    if not_covered_cases>0:
        accuracy_not_covered = accuracy_not_covered/not_covered_cases
        
    if mixed_cases>0:
        accuracy_mixed = accuracy_mixed/mixed_cases
        
        
    
    return not_covered_cases, mixed_cases, difficult_cases, easy_cases, easiness, accuracy_not_covered, accuracy_mixed, accuracy_easy, accuracy_difficult  


easiness = get_cases_number(subjects_in_groups)[4]
easiness_selected = [item for item in easiness if item!='not covered']

In [94]:
# -----------------------------------------
# Subjects in the rule space
# -----------------------------------------
clustering_data = pd.DataFrame(0, columns = all_groups_reduced.index, index = kihd_phenotypes.index)

for i in range(all_groups_reduced.shape[0]):
    for ind in all_groups_reduced.loc[i, 'indices']:
        clustering_data.loc[ind, i] = 1
        
clustering_data = clustering_data.loc[~(clustering_data==0).all(axis=1)]

from sklearn.cluster import KMeans

gene_groups_tsne = TSNE(n_components=2, metric='euclidean', perplexity=30.0).fit_transform(clustering_data) 

In [98]:
# -----------------------------------------
# Use SVM to draw a border between easy and difficult
# Use kmeans to find subclucters within easy and difficult groups
# -----------------------------------------

from sklearn.svm import SVC

X = gene_groups_tsne[np.array(easiness_selected) != 'ambiguous case']
y = np.array(easiness_selected)[np.array(easiness_selected) != 'ambiguous case']

model = SVC(C=1.0, kernel='rbf', degree=3, gamma=0.1)
model.fit(X, y)

easy_difficult_predictions = model.predict(X)

n_clusters_easy = 10
kmeans_easy = KMeans(n_clusters=n_clusters_easy, random_state=None).fit(gene_groups_tsne[np.array(easiness_selected) == 'easy case'])

n_clusters_difficult = 6
kmeans_difficult = KMeans(n_clusters=n_clusters_difficult, random_state=None).fit(gene_groups_tsne[np.array(easiness_selected) == 'difficult case'])

kmeans_labels = np.repeat(-1, gene_groups_tsne.shape[0])
kmeans_labels[np.where(np.array(easiness_selected) == 'easy case')] = kmeans_easy.predict(gene_groups_tsne[np.array(easiness_selected) == 'easy case'])
kmeans_labels[np.where(np.array(easiness_selected) == 'difficult case')] = kmeans_difficult.predict(gene_groups_tsne[np.array(easiness_selected) == 'difficult case']) + n_clusters_easy


In [ ]:
# -----------------------------------------
# Put the result into a color plot
# -----------------------------------------

h = .1    

x_min, x_max = gene_groups_tsne[:, 0].min() - 5, gene_groups_tsne[:, 0].max() + 5
y_min, y_max = gene_groups_tsne[:, 1].min() - 5, gene_groups_tsne[:, 1].max() + 5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z[np.where(Z == 'easy case')] = kmeans_easy.predict(np.c_[xx.ravel(), yy.ravel()][np.where(Z == 'easy case')])
Z[np.where(Z == 'difficult case')] = kmeans_difficult.predict(np.c_[xx.ravel(), yy.ravel()][np.where(Z == 'difficult case')]) + n_clusters_easy

Z = Z.astype('int')

Z = Z.reshape(xx.shape)

plt.subplots(figsize=(20,10))

plt.figure(1)
plt.clf()
plt.imshow(Z, interpolation='none',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.PiYG, alpha = 0.25, 
           aspect='auto', origin='lower')

colors = []
for item in easiness_selected:
    if item == 'ambiguous case':
        colors.append('black')
    elif item == 'easy case':
        colors.append('green')
    else:
        colors.append('red')
        
values = []
for item in easiness_selected:
    if item == 'ambiguous case':
        values.append(0)
    elif item == 'easy case':
        values.append(1)
    else:
        values.append(2)

import matplotlib.patches as mpatches

colormap = np.array(['black', 'green', 'red', 'blue'])

plt.scatter(gene_groups_tsne[:, 0], gene_groups_tsne[:, 1], c=colormap[values], s = 10, alpha = 0.5)

# Plot the centroids as a white X

centroids_kmeans_easy = kmeans_easy.cluster_centers_
centroids_kmeans_difficult = kmeans_difficult.cluster_centers_
centroids = np.concatenate([centroids_kmeans_easy, centroids_kmeans_difficult])

plt.scatter(centroids_kmeans_easy[:, 0], centroids_kmeans_easy[:, 1],
            marker='*', s=30, linewidths=3.,
            c=colormap[3], zorder=5)

plt.scatter(centroids_kmeans_difficult[:, 0], centroids_kmeans_difficult[:, 1],
            marker='*', s=30, linewidths=3.,
            c=colormap[3], zorder=5)

pop_a = mpatches.Patch(color='black', label='ambiguous case')
pop_b = mpatches.Patch(color='green', label='easy case')
pop_c = mpatches.Patch(color='red', label='difficult case')
pop_d = mpatches.Patch(color='blue', label='centroid')

plt.legend(handles = [pop_a, pop_b, pop_c, pop_d])

not_ambiguous_case = np.array(easiness_selected) != 'ambiguous case'

sick_tpr = np.array([])
healthy_tnr = np.array([])
for cl in range(n_clusters_easy + n_clusters_difficult):
    sick = kihd_outcomes.loc[clustering_data.loc[(kmeans_labels == cl) & not_ambiguous_case].index, 'cvd16'].sum()
    healthy = len(clustering_data.loc[(kmeans_labels == cl) & not_ambiguous_case].index) - sick
    
    items = kihd_outcomes.loc[clustering_data.loc[(kmeans_labels == cl) & not_ambiguous_case].index].index
    
    s_tpr = subjects_in_groups.loc[kihd_outcomes.loc[items][kihd_outcomes.loc[items, 'cvd16'] == 1].index, 'accuracy'].mean()*100
    h_tnr = subjects_in_groups.loc[kihd_outcomes.loc[items][kihd_outcomes.loc[items, 'cvd16'] == 0].index, 'accuracy'].mean()*100

    sick_tpr = np.append(sick_tpr, [sick, s_tpr])
    healthy_tnr = np.append(healthy_tnr, [healthy, h_tnr])
    
sick_tpr = sick_tpr.reshape(-1, 2)
healthy_tnr = healthy_tnr.reshape(-1, 2)

accuracy_clusters = np.array([])
for cl in range(n_clusters_easy + n_clusters_difficult):
    accuracy_clusters = np.append(accuracy_clusters, subjects_in_groups.loc[clustering_data.loc[kmeans_labels == cl].index, 'accuracy'].mean())

for cl in range(n_clusters_easy + n_clusters_difficult):
    plt.text(centroids[cl, 0], centroids[cl, 1] + 1, "{}%".format(round(accuracy_clusters[cl]*100, 2)),
             fontsize = 15, color='blue', 
             horizontalalignment='center', verticalalignment='bottom', style = 'normal')

for cl in range(n_clusters_easy + n_clusters_difficult):
    plt.text(centroids[cl, 0], centroids[cl, 1] - 2, "no cvd: {:.0f} -> {:.2f}%\ncvd: {:.0f} -> {:.2f}%".format(healthy_tnr[cl][0], healthy_tnr[cl][1],
             sick_tpr[cl][0], sick_tpr[cl][1]),
             fontsize = 10, color='blue', 
             horizontalalignment='center', verticalalignment='top', style = 'normal')

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()
